In [1]:
import tensorflow as tf
import numpy as np
import gym
import datetime
from tqdm import tqdm
import random
from go_ai import data, metrics, mcts, models, policies
import matplotlib.pyplot as plt
import os

# Hyperparameters

In [2]:
BOARD_SIZE = 7

In [3]:
ITERATIONS = 256
EPISODES_PER_ITERATION = 64
BATCH_SIZE = 32

In [4]:
NUM_EVAL_GAMES = 64

In [5]:
LEARNING_RATE = 2e-3

In [6]:
WEIGHTS_DIR = 'model_weights/'
CHECKPOINT_PATH = WEIGHTS_DIR + 'checkpoint_{}x{}.h5'.format(BOARD_SIZE, BOARD_SIZE)
TMP_WEIGHTS_PATH = WEIGHTS_DIR + 'tmp.h5'
LOAD_SAVED_MODELS = True

In [7]:
NUM_WORKERS = 4

In [8]:
EPISODES_DIR = './data/'

# Go Environment
Train on a small board with heuristic reward for fast training and efficient debugging

In [9]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Metrics and Tensorboard

In [10]:
!rm -rf ./logs/

Metrics

In [11]:
tb_metrics = {}
for metric_key in ['val_loss', 'overall_loss', 'move_loss']:
    tb_metrics[metric_key] = tf.keras.metrics.Mean('{}'.format(metric_key), 
                                                   dtype=tf.float32)
tb_metrics['pred_win_acc'] = tf.keras.metrics.Accuracy()

Tensorboard

In [12]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/actor_critic/{}/main'.format(current_time)
summary_writer = tf.summary.create_file_writer(log_dir)    

# Machine Learning Models

In [13]:
actor_critic = models.make_actor_critic(BOARD_SIZE, 'val_net', 'tanh')

KeyboardInterrupt: 

In [ ]:
_ = tf.keras.utils.plot_model(actor_critic, to_file='logs/model.png')

In [ ]:
actor_critic.summary()

In [ ]:
if LOAD_SAVED_MODELS:
    actor_critic.load_weights(CHECKPOINT_PATH)
    print("Loaded saved models")
else:
    actor_critic.save_weights(CHECKPOINT_PATH)
    print("Synced checkpoint with temp")
    
actor_critic.save_weights(TMP_WEIGHTS_PATH)

# Policies

In [ ]:
temp_policy_args = {
    'mode': 'actor_critic',
    'model_path': TMP_WEIGHTS_PATH,
}

In [ ]:
checkpoint_policy_args = {
    'mode': 'actor_critic',
    'model_path': CHECKPOINT_PATH,
}

In [ ]:
random_policy_args = {
    'mode': 'random'
}

In [ ]:
greedy_policy_args = {
    'mode': 'greedy'
}

# Value Function

In [ ]:
tmp_forward_func_args = {
    'weights_path': TMP_WEIGHTS_PATH
}

# Demo Trajectories

Symmetries

In [ ]:
%%time
metrics.plot_symmetries(go_env, actor_critic, 'logs/symmetries.jpg')

Plot a whole game trajectory

In [ ]:
%%time
data.make_episodes(BOARD_SIZE, temp_policy_args, temp_policy_args, tmp_forward_func_args, 1, num_workers=1)

In [ ]:
%%time
fig = metrics.gen_traj_fig(go_env, actor_critic)
fig.savefig('logs/a_trajectory.jpg')
plt.close()

# Train

In [ ]:
actor_critic_opt = tf.keras.optimizers.Adam(LEARNING_RATE)

In [ ]:
for iteration in range(ITERATIONS):
    for optimize_mode in ['critic', 'actor']:
        # Make and write out the episode data
        data.make_episodes(BOARD_SIZE, temp_policy_args, temp_policy_args, tmp_forward_func_args,
                           EPISODES_PER_ITERATION, num_workers=NUM_WORKERS, outdir=EPISODES_DIR)
        # Read in the episode data
        np_data = data.episodes_from_dir(EPISODES_DIR)
        batched_np_data = [np.array_split(datum, len(np_data[0]) // BATCH_SIZE) for datum in np_data]
        batched_mem = list(zip(*batched_np_data))

        # Optimize
        models.update_critic(actor_critic, optimize_mode, batched_mem, actor_critic_opt, iteration, tb_metrics)

        # Save weights for evaluation
        actor_critic.save_weights(TMP_WEIGHTS_PATH)
    
    # Evaluate against checkpoint model and other baselines
    rand_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, random_policy_args, 
                                       None, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
    greed_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, greedy_policy_args, 
                                        None, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
    opp_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, checkpoint_policy_args, 
                                      None, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)

    stats = "{:.1f}%O, {:.1f}%R, {:.1f}%G".format(100*opp_win_rate, 
                                                 100*rand_win_rate,
                                                 100*greed_win_rate)

    # If it's better than the checkpoint, update
    if opp_win_rate > 0.6:
        actor_critic.save_weights(CHECKPOINT_PATH)
        print(f"{stats} Accepted new model")
    elif opp_win_rate > 0.5:
        print(f"{stats} Continuing to train current weights")
    else:
        actor_critic.load_weights(CHECKPOINT_PATH)
        print(f"{stats} Rejected new model")

    # Sync temp with checkpoint
    actor_critic.save_weights(TMP_WEIGHTS_PATH)

    # Log results and resets the metrics
    metrics.log_to_tensorboard(summary_writer, tb_metrics, iteration, go_env, 
                               actor_critic, 'logs/a_trajectory.jpg')

# Evaluate

Play against our AI

In [ ]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)
data.pit(go_env, black_policy=opponent_policy, white_policy=human_policy)